In [3]:
import pandas as pd
import os

### Data Gathering
#### This section of the notebooks reads in the data files and stores them im pandas dataframes.
The dataframes frames in this section all have colums of [1960 - 2019] and rows for each state execpt for industy_gdp_by_state_df which goes from [1997-2020] 

In [37]:
csv_path = os.path.join(os.getcwd(), "data/csv")

In [38]:
#Read in all datasets here 

vehicle_registration_df = pd.read_csv(os.path.join(csv_path, "vehicle_registrations_by_state.csv"))
energy_consumption_per_real_gdp_df = pd.read_csv(os.path.join(csv_path, "energy_consumption_per_real_gdp.csv"))
current_dollar_gdp_df = pd.read_csv(os.path.join(csv_path, "Current_dollar_GDP.csv")) #in millions
total_consuption_df = pd.read_csv(os.path.join(csv_path, "total_consuption.csv")) #in billion Btu
industy_gdp_by_state_df = pd.read_csv(os.path.join(csv_path, "industy_gdp_by_state.csv"))
total_population_df = pd.read_csv(os.path.join(csv_path, "total_population.csv"))
real_gdp_df = pd.read_csv(os.path.join(csv_path, "real_GDP.csv")) #in millions

In [39]:
total_population_df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100
0,AK,229,238,246,256,263,271,271,278,285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AL,3274,3316,3323,3358,3395,3443,3464,3458,3446,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AR,1789,1806,1853,1875,1897,1894,1899,1901,1902,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AZ,1321,1407,1471,1521,1556,1584,1614,1646,1682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CA,15870,16497,17072,17668,18151,18585,18858,19176,19394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
